# Make Network Viz

 Pull data from hdfs/hive

In [1]:
import pyspark 
sc = pyspark.sql.SparkSession.Builder()\
        .master('yarn')\
        .appName('hive-test')\
        .enableHiveSupport()\
        .getOrCreate()

In [2]:
sc

# Individuals

In [4]:
sc.sql('SELECT * FROM pq_crp_indivs16').show(1)

+-----+-------------------+------------+-------------+---------+--------------------+------+--------+----------+------+------+--------+-----+-----+---------+----+---------+---------+------+------------------+--------------------+--------------------+------+
|Cycle|         FECTransID|   ContribID|      Contrib|  RecipID|             Orgname|UltOrg|RealCode|      Date|Amount|Street|    City|State|  Zip|RecipCode|Type|   CmteID|  OtherID|Gender|         Microfilm|          Occupation|            Employer|Source|
+-----+-------------------+------------+-------------+---------+--------------------+------+--------+----------+------+------+--------+-----+-----+---------+----+---------+---------+------+------------------+--------------------+--------------------+------+
| 2016|4081720161312405629|p0002929586 |CORMIER, KATE|N00000528|Community Resourc...|      |   Y4000|2016-05-03|    27|      |HATFIELD|   MA|01038|       DI| 15E|C00577130|C00401224|     F|201608159022288094|AUTISM SUPPORT BR.

In [5]:
sc.sql('SELECT ContribID,  Contrib, RecipID, Orgname, Gender, SUM(Amount) as contribution FROM pq_crp_indivs16 GROUP BY ContribID,  Contrib, RecipID, Orgname, Gender').show(1)

+------------+-------------+---------+--------------------+------+------------+
|   ContribID|      Contrib|  RecipID|             Orgname|Gender|contribution|
+------------+-------------+---------+--------------------+------+------------+
|p0003261456 |NORTON, AARON|N00000528|Northwestern Univ...|     M|          18|
+------------+-------------+---------+--------------------+------+------------+
only showing top 1 row



In [25]:
Individual_network_I = sc.sql('SELECT t1.Contrib, t1.Orgname, t1.Gender,\
                              t2.FirstLastP, t2.party, t3.ge_winner_ind_guess, SUM(t1.Amount) as contr_amt \
            FROM pq_crp_indivs16 t1\
            LEFT JOIN pq_crp_cands16 t2 \
            ON t1.RecipID= t2.CID  \
            LEFT JOIN pq_fec_elect t3 \
            ON (t2.feccandid = t3.fec_id) AND (t2.cycle = t3.cycle) \
            WHERE  t2.feccandid IS NOT NULL and t3.ge_winner_ind_guess IS NOT NULL\
            GROUP BY t1.ContribID,  t1.Contrib, t1.Orgname, t1.Gender, t2.FirstLastP, t2.party, t3.ge_winner_ind_guess')
Individual_network_I.show(10)

+--------------------+--------------------+------+------------------+-----+-------------------+---------+
|             Contrib|             Orgname|Gender|        FirstLastP|party|ge_winner_ind_guess|contr_amt|
+--------------------+--------------------+------+------------------+-----+-------------------+---------+
|   O'LEARY, JAYLEIGH|           Childcare|     F|Bernie Sanders (D)|    D|               null|      289|
|    SANDOVAL, MARCUS|    Decline To State|     M|Bernie Sanders (D)|    D|               null|      336|
|        MORAN, KEVIN|              Ingram|     M|Bernie Sanders (D)|    D|               null|      428|
|       JOHNSON, LUKE|     Central Casting|     M|Bernie Sanders (D)|    D|               null|      100|
|      PAUL, CRISTINA|  Harvard University|     F|Bernie Sanders (D)|    D|               null|       99|
|  PEARSON, MICHAEL M|Dartmouth College...|     M|Bernie Sanders (D)|    D|               null|      479|
|PETTEY, PATRICK J...|Allignment Govern...|   

In [4]:
Individual_network.count()

2802843

In [20]:
Individual_network_I.count()

1517301

In [26]:
df_individual_network = Individual_network_I.toPandas()
df_individual_network.head()

,Contrib,Orgname,Gender,FirstLastP,party,ge_winner_ind_guess,contr_amt
0,"O'LEARY, JAYLEIGH",Childcare,F,Bernie Sanders (D),D,null,289
1,"SANDOVAL, MARCUS",Decline To State,M,Bernie Sanders (D),D,null,336
2,"MORAN, KEVIN",Ingram,M,Bernie Sanders (D),D,null,428
3,"JOHNSON, LUKE",Central Casting,M,Bernie Sanders (D),D,null,100
4,"PAUL, CRISTINA",Harvard University,F,Bernie Sanders (D),D,null,99


In [27]:
df_individual_network.shape

(1517301, 7)

# Committees 

In [9]:
sc.sql('SELECT * FROM pq_crp_pacs16').show(1)

+-----+-------------------+---------+---------+------+----------+--------+----+---+---------+
|Cycle|           FECRecNo|    PACID|      CID|Amount|      Date|RealCode|Type| DI|FECCandID|
+-----+-------------------+---------+---------+------+----------+--------+----+---+---------+
| 2016|4102520161345215195|C00605212|N00033982|1700.0|2016-09-21|   J2200| 24K|  D|H2AZ08102|
+-----+-------------------+---------+---------+------+----------+--------+----+---+---------+
only showing top 1 row



In [18]:
sc.sql('SELECT * FROM pq_crp_cmtes16').show(1)

+-----+---------+--------------+---------+--------------+---------+---------+---------+-----+--------+------+---------+-------+------+
|Cycle|   CmteID|      PACShort|Affiliate|        Ultorg|  RecipID|RecipCode|FECCandID|Party|PrimCode|Source|Sensitive|Foreign|Active|
+-----+---------+--------------+---------+--------------+---------+---------+---------+-----+--------+------+---------+-------+------+
| 2016|C00000059|Hallmark Cards|         |Hallmark Cards|C00000059|       PB|         |     |   C1400| WebAM|        N|  false|     1|
+-----+---------+--------------+---------+--------------+---------+---------+---------+-----+--------+------+---------+-------+------+
only showing top 1 row



In [134]:
network_committee = sc.sql('SELECT t3.pacshort, t2.FirstLastP, t2.party, t4.ge_winner_ind_guess, t1.Type, SUM(t1.Amount) as contribution \
            FROM pq_crp_pacs16 t1 \
            LEFT JOIN pq_crp_cands16 t2 \
            ON t1.feccandid = t2.feccandid  \
            LEFT JOIN pq_crp_cmtes16 t3 \
            ON t1.PACID = t3.CmteID \
            LEFT JOIN pq_fec_elect t4 \
            ON t1.feccandid = t4.fec_id \
            WHERE t2.feccandid IS NOT NULL and t4.ge_winner_ind_guess IS NOT NULL \
            GROUP BY t1.PACID, t1.FECCandID, t3.pacshort, t2.FirstLastP, t2.party, t4.ge_winner_ind_guess, t1.Type')
network_committee.show(3)

+--------------------+--------------------+-----+-------------------+----+------------+
|            pacshort|          FirstLastP|party|ge_winner_ind_guess|Type|contribution|
+--------------------+--------------------+-----+-------------------+----+------------+
|Minn-Dak Farmers ...|   Steven A King (R)|    R|               null| 24K|      1000.0|
|Painters & Allied...|Richard Blumentha...|    D|                  W| 24K|      7500.0|
|  Polaris Industries|     Kristi Noem (R)|    R|                  W| 24K|     14000.0|
+--------------------+--------------------+-----+-------------------+----+------------+
only showing top 3 rows



In [135]:
df_committee_network = network_committee.toPandas()
df_committee_network.head()

,pacshort,FirstLastP,party,ge_winner_ind_guess,Type,contribution
0,Minn-Dak Farmers Co-Op,Steven A King (R),R,null,24K,1000.0
1,Painters & Allied Trades Union,Richard Blumenthal (D),D,W,24K,7500.0
2,Polaris Industries,Kristi Noem (R),R,W,24K,14000.0
3,American Physical Therapy Assn,Jim Langevin (D),D,W,24K,21000.0
4,General Motors,Bradley Byrne (R),R,null,24K,2500.0


# Industry

Industry is the most comprehensive view because it merges the contributions from individuals and committees

In [86]:
#just committees
industry_viz = sc.sql('SELECT firstlastp, t2.party, Catname AS Industry, ge_winner_ind_guess, sum(amount) AS contr_amt\
       FROM pq_crp_cands16 t2 \
       LEFT JOIN pq_crp_pacs16 t1 \
       ON t1.feccandid= t2.feccandid \
       INNER JOIN pq_crp_industry_names t3 \
       ON t1.realcode = t3.Catcode \
       LEFT JOIN pq_us_legislators t4 \
       ON t1.cid = t4.opensecrets_id \
       LEFT JOIN pq_fec_elect t5 \
       ON (t1.feccandid = t5.fec_id) AND (t2.cycle = t5.cycle) \
       WHERE di == "D" AND \
       realcode NOT LIKE "z9*%"  AND \
       realcode NOT LIKE "z4*%" AND t2.cycle = 2016 AND  ge_winner_ind_guess IS NOT NULL\
       GROUP BY firstlastp, t2.party, catname,ge_winner_ind_guess  ')
industry_viz.show(5)

+-------------------+-----+--------------------+-------------------+---------+
|         firstlastp|party|            Industry|ge_winner_ind_guess|contr_amt|
+-------------------+-----+--------------------+-------------------+---------+
|Gregory W Meeks (D)|    D|     Stock exchanges|                  W|  35000.0|
|  Eric Swalwell (D)|    D|            Dentists|                  W|  17500.0|
|  Jackie Speier (D)|    D|Other non-physici...|                  W|   4500.0|
|   Peter Roskam (R)|    R|Farm organization...|                  W|   4477.0|
|  Garret Graves (R)|    R|            Pro-Guns|                  W|      0.0|
+-------------------+-----+--------------------+-------------------+---------+
only showing top 5 rows



In [113]:
#Just individuals
industry_viz = sc.sql('SELECT t2.firstlastp, t2.party, t5.ge_winner_ind_guess, t4.RecipCode, t3.Catname AS Industry, sum(t4.amount) AS contr_amt \
       FROM pq_crp_cands16 t2 \
       LEFT JOIN pq_crp_indivs16 t4 \
       ON t2.CID = t4.RecipID \
       LEFT JOIN pq_crp_industry_names t3 \
       ON (t4.realcode = t3.catcode)\
       LEFT JOIN pq_fec_elect t5 \
       ON (t2.feccandid = t5.fec_id) AND (t4.cycle = t5.cycle)  \
       WHERE t4.realcode NOT LIKE "z9*%" and t4.realcode NOT LIKE "z4*%"\
        AND t2.cycle = 2016 AND  ge_winner_ind_guess IS NOT NULL \
       GROUP BY firstlastp, t2.party, t3.catname,t4.recipcode, ge_winner_ind_guess').show(2)

+-----------------+-----+-------------------+---------+---------------+---------+
|       firstlastp|party|ge_winner_ind_guess|RecipCode|       Industry|contr_amt|
+-----------------+-----+-------------------+---------+---------------+---------+
|Robert Derlet (D)|    D|               null|       DL|     Physicians|      650|
|    Ro Khanna (D)|    D|                  W|       DW|Venture capital|   382600|
+-----------------+-----+-------------------+---------+---------------+---------+
only showing top 2 rows



In [112]:
industry_viz = sc.sql('SELECT t4.RecipCode \
       FROM pq_crp_cands16 t2 \
       LEFT JOIN pq_crp_indivs16 t4 \
       ON t2.CID = t4.RecipID \
       LEFT JOIN pq_crp_industry_names t3 \
       ON (t4.realcode = t3.catcode)\
       LEFT JOIN pq_fec_elect t5 \
       ON (t2.feccandid = t5.fec_id) AND (t4.cycle = t5.cycle)  \
       GROUP BY t4.recipcode ').show()

+---------+
|RecipCode|
+---------+
|       RO|
|       DC|
|       3N|
|     null|
|       RI|
|       DN|
|       DL|
|       RC|
|       3L|
|       DI|
|       3I|
|       RW|
|       RL|
|       UN|
|       DO|
|       3W|
|       RN|
|       RK|
|       DW|
+---------+



In [130]:
industry_viz = sc.sql('select * from (\
       SELECT t2.firstlastp, t2.party, t5.ge_winner_ind_guess,  t3.Catname AS Industry, sum(t4.amount) AS contr_amt \
           FROM pq_crp_cands16 t2 \
           LEFT JOIN pq_crp_indivs16 t4 \
           ON t2.CID = t4.RecipID \
           LEFT JOIN pq_crp_industry_names t3 \
           ON (t4.realcode = t3.catcode)\
           LEFT JOIN pq_fec_elect t5 \
           ON (t2.feccandid = t5.fec_id) AND (t4.cycle = t5.cycle)  \
           WHERE t4.realcode NOT LIKE "z9*%" and t4.realcode NOT LIKE "z4*%"\
           AND t2.cycle = 2016 AND  ge_winner_ind_guess IS NOT NULL \
           GROUP BY firstlastp, t2.party, t3.catname, ge_winner_ind_guess \
       UNION ALL\
       SELECT t2.firstlastp, t2.party,t5.ge_winner_ind_guess, Catname AS Industry, sum(amount) AS contr_amt\
           FROM pq_crp_cands16 t2 \
           LEFT JOIN pq_crp_pacs16 t1 \
           ON t1.feccandid= t2.feccandid \
           INNER JOIN pq_crp_industry_names t3 \
           ON t1.realcode = t3.Catcode \
           LEFT JOIN pq_us_legislators t4 \
           ON t1.cid = t4.opensecrets_id \
           LEFT JOIN pq_fec_elect t5 \
           ON (t1.feccandid = t5.fec_id) AND (t2.cycle = t5.cycle) \
           WHERE di == "D" AND \
           t1.realcode NOT LIKE "z9*%"  AND\
           t1.realcode NOT LIKE "z4*%" AND t2.cycle = 2016 AND  t5.ge_winner_ind_guess IS NOT NULL \
           GROUP BY t2.firstlastp, t2.party, catname, t5.ge_winner_ind_guess) ')
industry_viz.show(5)

+--------------------+-----+-------------------+--------------------+---------+
|          firstlastp|party|ge_winner_ind_guess|            Industry|contr_amt|
+--------------------+-----+-------------------+--------------------+---------+
|Catherine Cortez ...|    D|                  W|Transfer from int...|5737117.0|
| Carlos Pesquera (3)|    3|               null|             Retired|  18560.0|
|   Denise Juneau (D)|    D|               null|             Resorts|    229.0|
|    Will Jawando (D)|    D|               null|Corporate lawyers...|   -200.0|
|        Judy Chu (D)|    D|                  W|  Democratic/Liberal|   4300.0|
+--------------------+-----+-------------------+--------------------+---------+
only showing top 5 rows



In [131]:
df_industry_viz = industry_viz.toPandas()

In [132]:
df_industry_viz.tail()

,firstlastp,party,ge_winner_ind_guess,Industry,contr_amt
128245,Bill Flores (R),R,W,Building operators and managers,5000.0
128246,Mike Crane (R),R,null,Republican/Conservative,5000.0
128247,Tom O'Halleran (D),D,W,Other unions,2500.0
128248,John Carter (R),R,W,Food wholesalers,1000.0
128249,Carol Shea-Porter (D),D,W,"Defense policy, doves",10000.0


# Postgresql

Make connection to Postgresql to hold data --Thanks Matt for the tutorial here :)

In [7]:
import sqlite3
import psycopg2
from psycopg2.extras import execute_values
import pandas as pd
import base64
import os
import re
import sys
from IPython.display import display, HTML
import time
import tqdm

USE_POSTGRES=True

if not USE_POSTGRES:
    import sqlite3
if USE_POSTGRES:
    import psycopg2
    import psycopg2.extras
from flask import Flask
from flask import g

app = Flask(__name__)
DATABASE = "/home/daniel/kingmakers_capstone/data/os.db"
CONNECTION_STR = "/var/www/production/connection_str"
ROOT_CERT = "/var/www/production/root.crt"


if USE_POSTGRES:
    def get_db():
        with open(CONNECTION_STR, 'r') as f:
            conx = f.read().strip()
        print("connecting to postgres...")
        db =  psycopg2.connect(conx,
                                            sslrootcert=ROOT_CERT,
                                            cursor_factory=psycopg2.extras.RealDictCursor)
        return db    

/opt/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [8]:
pg_conn_str = get_db()

connecting to postgres...


OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.


In [143]:
pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';", pg_conn_str)

,table_name
0,cand
1,cmte
2,network_industry
3,expends
4,cands
5,indivs
6,pacs2cands
7,cmtes
8,pacs2pacs
9,industry


# Individuals

Make table schema for postgresql

In [139]:
network_individ = str('CREATE TABLE network_individ ( \
        contr_amt DOUBLE PRECISION , \
        firstlastp  VARCHAR(90), \
        party VARCHAR(2), \
        Gender VARCHAR(2), \
        Contrib VARCHAR(90), \
        Orgname VARCHAR(90), \
        ge_winner_ind_guess VARCHAR(15))')
network_individ

'CREATE TABLE network_individ (         contr_amt DOUBLE PRECISION ,         firstlastp  VARCHAR(90),         party VARCHAR(2),         Gender VARCHAR(2),         Contrib VARCHAR(90),         Orgname VARCHAR(90),         ge_winner_ind_guess VARCHAR(15))'

In [137]:
tbl_row_df = list(df_individual_network.columns.values)
cols = ','.join(tbl_row_df)
print(cols)

Contrib,Orgname,Gender,FirstLastP,party,ge_winner_ind_guess,contr_amt


Migrate data

In [146]:
tbl = 'network_individ'

pg_create_table = network_individ
print(pg_create_table)

with pg_conn_str.cursor() as pgcur:
    pgcur.execute(f'DROP TABLE IF EXISTS {tbl}')
    pgcur.execute(pg_create_table)
    
    start_time = time.time()
    
    tbl_row_df = list(df_individual_network.columns.values)
    cols = ','.join(tbl_row_df)
    print(cols)

    vals =  df_individual_network.values #.iloc[1:6,1:10]
    execute_values(pgcur,
                   f"""INSERT INTO {tbl}({cols}) VALUES %s""", vals ) #,page_size=1000)

    end_time = time.time()
    print(f'inserted rows in {round((end_time - start_time)*1000, 2)}ms')
pg_conn_str.commit()

CREATE TABLE network_industry (         contr_amt DOUBLE PRECISION ,         feccandid VARCHAR(15),         bioguide_id VARCHAR(20),         firstlastp  VARCHAR(90),         party VARCHAR(2),         Industry VARCHAR(50),         ge_winner_ind_guess VARCHAR(15))
contr_amt,feccandid,bioguide_id,firstlastp,party,Industry,ge_winner_ind_guess
inserted rows in 4326.45ms


In [147]:
pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';", pg_conn_str)

,table_name
0,network_industry
1,cand
2,cmte
3,expends
4,cands
5,indivs
6,pacs2cands
7,cmtes
8,pacs2pacs
9,industry


# Industry

In [140]:
network_industry = str('CREATE TABLE network_industry ( \
        contr_amt DOUBLE PRECISION , \
        firstlastp  VARCHAR(90), \
        party VARCHAR(2), \
        Industry VARCHAR(50), \
        ge_winner_ind_guess VARCHAR(15))')
network_industry

'CREATE TABLE network_industry (         contr_amt DOUBLE PRECISION ,         firstlastp  VARCHAR(90),         party VARCHAR(2),         Industry VARCHAR(50),         ge_winner_ind_guess VARCHAR(15))'

In [141]:
tbl_row_df = list(df_industry_viz.columns.values)
cols = ','.join(tbl_row_df)
print(cols)

firstlastp,party,ge_winner_ind_guess,Industry,contr_amt


In [146]:
tbl = 'network_industry'

pg_create_table = network_industry
print(pg_create_table)

with pg_conn_str.cursor() as pgcur:
    pgcur.execute(f'DROP TABLE IF EXISTS {tbl}')
    pgcur.execute(pg_create_table)
    
    start_time = time.time()
    
    tbl_row_df = list(df_industry_viz.columns.values)
    cols = ','.join(tbl_row_df)
    print(cols)

    vals =  df_industry_viz.values #.iloc[1:6,1:10]
    execute_values(pgcur,
                   f"""INSERT INTO {tbl}({cols}) VALUES %s""", vals ) #,page_size=1000)

    end_time = time.time()
    print(f'inserted rows in {round((end_time - start_time)*1000, 2)}ms')
pg_conn_str.commit()

CREATE TABLE network_industry (         contr_amt DOUBLE PRECISION ,         feccandid VARCHAR(15),         bioguide_id VARCHAR(20),         firstlastp  VARCHAR(90),         party VARCHAR(2),         Industry VARCHAR(50),         ge_winner_ind_guess VARCHAR(15))
contr_amt,feccandid,bioguide_id,firstlastp,party,Industry,ge_winner_ind_guess
inserted rows in 4326.45ms


In [147]:
pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';", pg_conn_str)

,table_name
0,network_industry
1,cand
2,cmte
3,expends
4,cands
5,indivs
6,pacs2cands
7,cmtes
8,pacs2pacs
9,industry


# committee

In [142]:
committee_network = str('CREATE TABLE committee_network  ( \
        contr_amt DOUBLE PRECISION , \
        firstlastp  VARCHAR(90), \
        party VARCHAR(2), \
        pacshort VARCHAR(50), \
        type VARCHAR(10), \
        ge_winner_ind_guess VARCHAR(15))')
committee_network 

'CREATE TABLE committee_network  (         contr_amt DOUBLE PRECISION ,         firstlastp  VARCHAR(90),         party VARCHAR(2),         pacshort VARCHAR(50),         type VARCHAR(10),         ge_winner_ind_guess VARCHAR(15))'

In [143]:
tbl_row_df = list(df_committee_network.columns.values)
cols = ','.join(tbl_row_df)
print(cols)

pacshort,FirstLastP,party,ge_winner_ind_guess,Type,contribution


In [146]:
tbl = 'committee_network'

pg_create_table = committee_network 
print(pg_create_table)

with pg_conn_str.cursor() as pgcur:
    pgcur.execute(f'DROP TABLE IF EXISTS {tbl}')
    pgcur.execute(pg_create_table)
    
    start_time = time.time()
    
    tbl_row_df = list(df_committee_network.values)
    cols = ','.join(tbl_row_df)
    print(cols)

    vals =  df_committee_network.values #.iloc[1:6,1:10]
    execute_values(pgcur,
                   f"""INSERT INTO {tbl}({cols}) VALUES %s""", vals ) #,page_size=1000)

    end_time = time.time()
    print(f'inserted rows in {round((end_time - start_time)*1000, 2)}ms')
pg_conn_str.commit()

CREATE TABLE network_industry (         contr_amt DOUBLE PRECISION ,         feccandid VARCHAR(15),         bioguide_id VARCHAR(20),         firstlastp  VARCHAR(90),         party VARCHAR(2),         Industry VARCHAR(50),         ge_winner_ind_guess VARCHAR(15))
contr_amt,feccandid,bioguide_id,firstlastp,party,Industry,ge_winner_ind_guess
inserted rows in 4326.45ms


In [147]:
pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';", pg_conn_str)

,table_name
0,network_industry
1,cand
2,cmte
3,expends
4,cands
5,indivs
6,pacs2cands
7,cmtes
8,pacs2pacs
9,industry
